<a href="https://colab.research.google.com/github/yleessam/2501_dl_env_colab/blob/main/9_1_object_detecting_yolov3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정지 영상에서 객체 탐지

## 필요한 라이브러리 import

In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow # Google Colab에서 이미지를 표시하기 위한 함수

## YOLO(YOu Only Look Once) 모델 관련 파일 다운로드

In [ ]:
# YOLO(YOu Only Look Once) 모델
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O coco.names

## YOLO 모델 초기화 및 로드 함수

In [ ]:
def load_yolo():
    net = cv.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers

## 이미지 로드 및 전처리 함수

In [ ]:
def load_image(img_path):
    img = cv.imread(img_path)
    img = cv.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    return img, height, width, channels

## 객체 탐지 함수

In [ ]:
def detect_objects(img, net, outputLayers):
    blob = cv.dnn.blobFromImage(img, scalefactor=0.00392, size=(416, 416), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(outputLayers)
    return blob, outputs


## 탐지된 객체 정보


YOLO 네트워크의 출력에서 객체의 위치와 해당 클래스의 신뢰도를 추출하여 리스트로 저장

YOLO 모델의 출력을 파싱하여 각 객체에 대한 경계 상자의 위치, 신뢰도 점수 및 클래스 ID를 추출합니다. 이 정보는 객체를 이미지 위에 표시하거나 추가 분석을 위해 사용

In [ ]:
def get_box_dimensions(outputs, height, width):
    boxes = []
    confs = []
    class_ids = []

    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confs.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confs, class_ids


## NMS & 바운딩 박스 그리기 함수

In [ ]:
def draw_labels(boxes, confs, colors, class_ids, classes, img):
    indexes = cv.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confs[i]
            color = colors[class_ids[i]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, f"{label} {confidence:.2f}", (x, y + 30), font, 1, color, 2)
    cv2_imshow(img)

## 객체 탐지 실행


구글 이미지 검색 > 마우스 우클릭 > 링크주소 복사 실행

In [ ]:
import requests

# 이미지 URL
image_url = "https://images.squarespace-cdn.com/content/v1/56690c0ca128e6e00f021af6/1461148473502-9D3SLDUVC7J87LTG63SL/BEAD+Al+Bateen+street2.JPG"

# 저장할 파일 이름
output_file = "street2.jpg"

# 이미지 다운로드 및 저장
response = requests.get(image_url, stream=True)
if response.status_code == 200:
    with open(output_file, "wb") as file:
        file.write(response.content)
    print(f"이미지가 {output_file}로 저장되었습니다!")
else:
    print(f"이미지를 다운로드할 수 없습니다. 상태 코드: {response.status_code}")


이미지가 street2.jpg로 저장되었습니다!


In [ ]:
net, classes, colors, output_layers = load_yolo()

image, height, width, channels = load_image("street2.jpg")

blob, outputs = detect_objects(image, net, output_layers)

boxes, confs, class_ids = get_box_dimensions(outputs, height, width)

draw_labels(boxes, confs, colors, class_ids, classes, image)
